In [1]:
from decoders.interleaved import read_pcap
from utils.matlab import db
from tensorflow.keras import Model
from tensorflow.keras import Input
from tensorflow.keras.layers import Rescaling
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import LSTM
from tensorflow.keras.models import Sequential
import glob
from numpy import inf
from tensorflow.keras.layers import Bidirectional


import numpy as np
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping
import os

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, StratifiedKFold

from scipy.interpolate import interp1d

In [60]:
base_dir = "."
dataset_dir = os.path.join("data")
test_dir = os.path.join("generated/data")

In [3]:
FS = 100
ALL_ACTIVITIES = ["static", "standing", "walking", "falling"]

In [4]:
def loadFromDat(inputFile, windowSize = FS, step = 50):
#     print(int((csi_data.nsamples-1)*(100/average_sample_rate)+1))
    
    csi_data = read_pcap(inputFile)
    csi = csi_data.csi
    
    
    first_timestamp = float(csi_data.timestamps[0])
    last_timestamp = float(csi_data.timestamps[-1])
    final_timestamp = last_timestamp-first_timestamp
    average_sample_rate = csi_data.nsamples/final_timestamp
    
    interp_func=interp1d(csi_data.timestamps, csi_data.csi, kind='linear', axis=0, fill_value="extrapolate")
    t_new = np.linspace(first_timestamp, last_timestamp, (csi_data.nsamples-1)*int((100/average_sample_rate)+1))
    csi_interp = interp_func(t_new)
    csi=csi_interp
    
    csi = db(np.abs(csi))
    finalData = csi[:, :, 0]
    finalData = np.transpose(finalData)
    
    new_average_sample_rate = len(csi_interp)/final_timestamp
    
    if new_average_sample_rate > FS:
        downsample_factor = int(new_average_sample_rate/FS)
        csi = csi[::downsample_factor]
    index = 0
    positiveInput = []
    
    while index + windowSize <= csi.shape[0]:
        curFeature = np.zeros((1, windowSize, 256))
        curFeature[0] = csi[index:index+windowSize, :].reshape(100,256)
        positiveInput.append(curFeature)
        index += step
    try:
        positiveInput = np.concatenate(positiveInput, axis = 0)
    except ValueError as e:
        positiveInput = np.zeros((1, windowSize, 256))
    positiveInput[positiveInput == -inf] = 0
    
    return positiveInput

In [39]:
def loadByActivityLabel(directory, activity, activities=ALL_ACTIVITIES, windowSize=FS, step = 50, downsample=1):
    activity = activity.lower()
    if activity not in activities:
        print("invalid activity: " + activity)
    
    dataPathPattern = os.path.join(directory,"{}*.pcap".format(activity))
    inputFiles = sorted(glob.glob(dataPathPattern, recursive =True))
    inputWindows = []
    index = 0
    for inputFile in inputFiles:
        index += 1
        csiOutput = loadFromDat(inputFile, windowSize = windowSize, step=step)
        if csiOutput is not None:
            inputWindows.append(csiOutput)
    inputArray = np.concatenate(inputWindows, axis = 0)
    outputLabels = np.zeros((inputArray.shape[0], len(activities)))
    outputLabels[:, activities.index(activity)] = 1
    return inputArray, outputLabels


In [63]:
def loadDataForActivities(directory, activities=ALL_ACTIVITIES, windowSize=100, step=50, downsample=1):
    x_all = []
    y_all = []
    for activity in activities:
        print(activity)
        inputArray, outputLabels = loadByActivityLabel(directory, activity, activities, windowSize=windowSize, step=step, downsample=downsample)
        x_all.append(inputArray)
        y_all.append(outputLabels)
    x_all = np.concatenate(x_all, axis=0)
    y_all = np.concatenate(y_all, axis=0)
    return (x_all, y_all)

In [45]:
x_all, y_all = loadDataForActivities(dataset_dir, downsample=1)

[[0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0.

In [8]:
loadDataForActivities(dataset_dir, downsample=1)

(array([[[75.82117487, 91.1968139 , 42.14419939, ..., 24.4870632 ,
          23.97940009, 17.63427994],
         [75.82117487, 91.1968139 , 42.14419939, ..., 25.13374666,
          18.82639991, 23.98065317],
         [75.82117487, 91.1968139 , 42.14419939, ..., 25.95279419,
          15.27814445, 23.73498563],
         ...,
         [75.82117487, 91.18616539, 42.14419939, ..., 24.09829638,
          20.2568525 , 20.38573453],
         [75.82117487, 91.19448347, 42.14419939, ..., 27.89504509,
          29.00928825, 22.21924811],
         [75.82117487, 91.1968139 , 42.14419939, ..., 27.52979228,
          27.65363008, 10.51237856]],
 
        [[75.82117487, 91.1968139 , 42.14419939, ..., 14.76296322,
          15.05162086, 15.90466465],
         [75.82117487, 91.1968139 , 42.14419939, ..., 21.79406852,
          12.90583814,  6.44155676],
         [75.82117487, 91.1968139 , 42.14419939, ..., 22.47093636,
          -1.78505337, 10.7315378 ],
         ...,
         [75.82117487, 91.1968139

In [36]:
x_train, x_val, y_train, y_val = train_test_split(x_all, y_all, stratify=y_all, test_size = 0.3, random_state=42)
print(len(x_val[0][3]))

256


In [10]:
batch_size = 256
epochs = 100

f1s = []
confs = []
y_val_total = np.array([])
y_pred_total = np.array([])


In [11]:
model = Sequential()
model.add(Conv1D(filters=128, kernel_size=5, activation='relu', strides=1))
model.add(MaxPooling1D(pool_size=4))
# model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
# model.add(MaxPooling1D(pool_size=4))
model.add(Bidirectional(LSTM(units=200, return_sequences=False)))
# model.add(Bidirectional(LSTM(units=200, return_sequences=False)))
model.add(Flatten())
model.add(Dense(4, activation='softmax'))


In [12]:
model.compile(optimizer=RMSprop(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])

In [13]:
model.fit(x_train, y_train, batch_size=256, epochs=100, shuffle=False, validation_data=(x_val, y_val))

Epoch 1/100
5/5 [==============================] - 16s 1s/step - loss: 1.3261 - accuracy: 0.4058 - val_loss: 1.0839 - val_accuracy: 0.6139
Epoch 2/100
5/5 [==============================] - 3s 620ms/step - loss: 1.0489 - accuracy: 0.6239 - val_loss: 1.0341 - val_accuracy: 0.4713
Epoch 3/100
5/5 [==============================] - 3s 625ms/step - loss: 0.9559 - accuracy: 0.6290 - val_loss: 0.9498 - val_accuracy: 0.5901
Epoch 4/100
5/5 [==============================] - 3s 595ms/step - loss: 1.0129 - accuracy: 0.5671 - val_loss: 0.9298 - val_accuracy: 0.6119
Epoch 5/100
5/5 [==============================] - 3s 634ms/step - loss: 0.8274 - accuracy: 0.7224 - val_loss: 0.8356 - val_accuracy: 0.7228
Epoch 6/100
5/5 [==============================] - 3s 591ms/step - loss: 0.7992 - accuracy: 0.7088 - val_loss: 0.8467 - val_accuracy: 0.6673
Epoch 7/100
5/5 [==============================] - 3s 666ms/step - loss: 0.7573 - accuracy: 0.7767 - val_loss: 0.7039 - val_accuracy: 0.8198
Epoch 8/100
5/5

Epoch 59/100
5/5 [==============================] - 3s 566ms/step - loss: 0.3715 - accuracy: 0.8472 - val_loss: 0.3226 - val_accuracy: 0.8317
Epoch 60/100
5/5 [==============================] - 3s 568ms/step - loss: 0.3657 - accuracy: 0.8328 - val_loss: 0.3826 - val_accuracy: 0.8376
Epoch 61/100
5/5 [==============================] - 3s 541ms/step - loss: 0.3934 - accuracy: 0.8514 - val_loss: 0.3469 - val_accuracy: 0.8297
Epoch 62/100
5/5 [==============================] - 3s 583ms/step - loss: 0.3708 - accuracy: 0.8217 - val_loss: 0.3406 - val_accuracy: 0.8515
Epoch 63/100
5/5 [==============================] - 3s 599ms/step - loss: 0.2464 - accuracy: 0.9372 - val_loss: 0.2637 - val_accuracy: 0.8931
Epoch 64/100
5/5 [==============================] - 3s 552ms/step - loss: 0.4109 - accuracy: 0.8158 - val_loss: 0.3545 - val_accuracy: 0.8752
Epoch 65/100
5/5 [==============================] - 3s 569ms/step - loss: 0.3498 - accuracy: 0.8633 - val_loss: 0.2701 - val_accuracy: 0.9129
Epoch 

In [59]:
y_pred = model.predict(x_val)
print(len(x_val[50][99]))

16/16 [==============================] - 1s 41ms/step
256


In [15]:
y_val_single = np.argmax(y_val, axis=1)
y_pred_single = np.argmax(y_pred, axis=1)

In [17]:
print(confusion_matrix(y_val_single, y_pred_single))
print(classification_report(y_val_single, y_pred_single, target_names=["static", "standing", "walking", "falling"]))

[[161   0   3   0]
 [  0  70  16   0]
 [  5   3 158   0]
 [  0   0   0  89]]
              precision    recall  f1-score   support

      static       0.97      0.98      0.98       164
    standing       0.96      0.81      0.88        86
     walking       0.89      0.95      0.92       166
     falling       1.00      1.00      1.00        89

    accuracy                           0.95       505
   macro avg       0.96      0.94      0.94       505
weighted avg       0.95      0.95      0.95       505



In [18]:
model.save("best_network.h5")

In [ ]:
# x_all = loadFromDat(f"./{test_dir}/falling.pcap")

FileNotFoundError: [WinError 3] The system cannot find the path specified: './generated/data/falling.pcap'